In [1]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git


  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-_y93lnmu/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-_y93lnmu/python-terrier
  Created wheel for python-terrier: filename=python_terrier-0.4.0-cp37-none-any.whl size=78065 sha256=ce3680509428fdd06d206f94c24ed68cee8f312d6906a19d986bad21ea915bc2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ufrootje/wheels/cc/bb/69/836d846a92c787b35ca6478119c0033762ab2b95d866eeb288
Successfully built python-terrier
  Found existing installation: python-terrier 0.4.0
    Uninstalling python-terrier-0.4.0:
      Successfully uninstalled python-terrier-0.4.0
  Cloning https://github.com/terrierteam/pyterrier_doc2query.git to /tmp/pip-req-build-o00rtsgp
  Running command git clone -q https://github.com/terrierteam/pyterrier_doc2query.git /tmp/pip-req-build-o00rtsgp
  Created wheel for pyterrier-doc2query: filename=pyterrier_doc2query-0.0.1-cp37-none-any.whl size=3842 

In [2]:
import os
if not os.path.exists("t5-base.zip"):
  !wget https://git.uwaterloo.ca/jimmylin/doc2query-data/raw/master/T5-passage/t5-base.zip
  !unzip t5-base.zip


In [3]:
import pyterrier as pt
pt.init()

PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [4]:
from pyterrier_doc2query import Doc2Query
doc2query = Doc2Query(out_attr="text", batch_size=8)


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Doc2query using cuda


In [5]:
sample_doc = "The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated"
doc2query._doc2query([sample_doc, sample_doc])

['why is the manhattan project important what were the main benefits the manhattan project had? why was the manhattan project so successful',
 'why did the manhattan project really happen what was the purpose of the manhattan project what is the most important thing the manhattan project accomplished']

In [6]:
import pandas as pd
df = pd.DataFrame([{'docno' : 'd1', 'text' : sample_doc}])
df

,docno,text
0,d1,The presence of communication amid scientific ...


In [7]:
doc2query.transform(df)

,docno,text
0,d1,why was the manhattan project important? why w...


In [8]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./index_path"
indexer = doc2query >> pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())

[INFO] [starting] building docstore
[INFO] If you have a local copy of http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/23e5607081191b153738e81fbd834680
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter: 0it [00:00, ?it/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.8%| 16.4k/2.13M [00:00<00:16, 128kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:11, 186kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 3.9%| 81.9k/2.13M [00:00<00:09, 205kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 6.9%| 147k/2.13M [00:00<00:07, 275kB/s] 
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 14.6%| 311k/2.13M [00:00<00:03, 

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [9]:
pt.Experiment([
  pt.BatchRetrieve(pt.get_dataset("vaswani").get_index(), wmodel="BM25"),
  pt.BatchRetrieve(indexref, wmodel="BM25"),
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  eval_metrics=["official"]
)

data.direct.bf:   0%|          | 0.00/327k [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/190k [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/301k [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/651k [00:00<?, ?iB/s]

data.lexicon.fsomaphash:   0%|          | 0.00/777 [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/89.3k [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/168k [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/882 [00:00<?, ?iB/s]

data.meta-0.fsomapfile:   0%|          | 0.00/725k [00:00<?, ?iB/s]

,name,num_q,num_ret,num_rel,num_rel_ret,map,gm_map,Rprec,bpref,recip_rank,iprec_at_recall_0.00,iprec_at_recall_0.10,iprec_at_recall_0.20,iprec_at_recall_0.30,iprec_at_recall_0.40,iprec_at_recall_0.50,iprec_at_recall_0.60,iprec_at_recall_0.70,iprec_at_recall_0.80,iprec_at_recall_0.90,iprec_at_recall_1.00,P_5,P_10,P_15,P_20,P_30,P_100,P_200,P_500,P_1000
0,BR(BM25),93.0,91930.0,2083.0,1937.0,0.296517,-1.621619,0.303662,0.934607,0.725665,0.751261,0.658588,0.523882,0.411899,0.345536,0.274788,0.207526,0.159466,0.105463,0.057922,0.023892,0.460215,0.352688,0.302509,0.269892,0.236918,0.126667,0.077903,0.037978,0.020828
1,BR(BM25),93.0,80737.0,2083.0,1450.0,0.166085,-2.374494,0.189872,0.706099,0.608390,0.630336,0.472489,0.302458,0.221471,0.163347,0.111102,0.075243,0.047604,0.023271,0.010832,0.008151,0.324731,0.251613,0.213620,0.189247,0.160932,0.081935,0.051613,0.026516,0.015591
